In [1]:
import os

In [2]:
import hashlib

In [3]:
import re

In [4]:
import cloudpickle

In [5]:
import time

In [6]:
def get_hash(binary):
    binary_no_ipykernel = re.sub(b"(?<=/ipykernel_)(.*)(?=/)", b"", binary)
    return str(hashlib.md5(binary_no_ipykernel).hexdigest())

In [7]:
def serialize_funct(fn, *args, **kwargs):
    binary = cloudpickle.dumps({"fn": fn, "args": args, "kwargs": kwargs})
    return {fn.__name__ + get_hash(binary=binary): binary}

In [8]:
def get_file_name(name, state):
    return name + "." + state + ".pl"

In [9]:
def write_to_file(funct_dict, state):
    file_name_lst = []
    for k, v in funct_dict.items():
        file_name = get_file_name(name=k, state=state)
        file_name_lst.append(file_name)
        with open(file_name, "wb") as f:
            f.write(v)
    return file_name_lst

In [10]:
def read_from_file(file_name):
    name = file_name.split(".")[0]
    with open(file_name, "rb") as f:
        return {name: f.read()}

In [11]:
def deserialize(funct_dict):
    return {k: cloudpickle.loads(v) for k, v in funct_dict.items()}

In [12]:
def apply_funct(apply_dict):
    return {k: v["fn"].__call__(*v["args"], **v["kwargs"]) for k, v in apply_dict.items()} 

In [13]:
def serialize_result(result_dict):
    return {k: cloudpickle.dumps(v) for k, v in result_dict.items()}

In [14]:
def execute_function(fn, *args, **kwargs):
    funct_dict = serialize_funct(fn, *args, **kwargs)
    file_lst = os.listdir()
    file_dict = {k: [f for f in file_lst if k in f] for k in funct_dict.keys()}
    for k, v in file_dict.items():
        if len(v) == 2: 
            file_name_result = [f for f in v if f.endswith("out.pl")][0]   
        else:
            if len(v) == 0:
                file_name_funct = write_to_file(
                    funct_dict=serialize_funct(fn, *args, **kwargs), 
                    state="in"
                )[0]
            else:
                file_name_funct = v[0]
            funct_dict = read_from_file(file_name=file_name_funct)
            apply_dict = deserialize(funct_dict=funct_dict)
            result_dict = apply_funct(apply_dict=apply_dict)
            write_to_file(funct_dict=serialize_result(result_dict=result_dict), state="out")
            file_name_result = file_name_funct.replace("in.pl", "out.pl")
        return list(deserialize(funct_dict=read_from_file(file_name=file_name_result)).values())[0]

# Write function to file 

In [15]:
def test_funct(a, b):
    time.sleep(5)
    return a+b

In [16]:
execute_function(test_funct, 1, 4)

5

In [17]:
os.listdir()  # test_funct93f7c993ad3e89a2fb17716ef985a8a6.out.pl

['store_function_v4.ipynb',
 'store_function_v6.ipynb',
 'store_function_v2.ipynb',
 'store_function_v5.ipynb',
 'store_function_v1.ipynb',
 'store_function_v3.ipynb',
 'test_funct93f7c993ad3e89a2fb17716ef985a8a6.in.pl',
 'test_funct93f7c993ad3e89a2fb17716ef985a8a6.out.pl',
 '.ipynb_checkpoints']

In [18]:
execute_function(test_funct, 1, 2)

3

In [19]:
os.listdir()

['store_function_v4.ipynb',
 'test_funct9af832e861123b84e12ff5dea30068af.out.pl',
 'store_function_v6.ipynb',
 'store_function_v2.ipynb',
 'store_function_v5.ipynb',
 'store_function_v1.ipynb',
 'store_function_v3.ipynb',
 'test_funct93f7c993ad3e89a2fb17716ef985a8a6.in.pl',
 'test_funct93f7c993ad3e89a2fb17716ef985a8a6.out.pl',
 'test_funct9af832e861123b84e12ff5dea30068af.in.pl',
 '.ipynb_checkpoints']

In [20]:
for f in os.listdir():
    if f.endswith("out.pl"):
        os.remove(f)

In [21]:
os.listdir()

['store_function_v4.ipynb',
 'store_function_v6.ipynb',
 'store_function_v2.ipynb',
 'store_function_v5.ipynb',
 'store_function_v1.ipynb',
 'store_function_v3.ipynb',
 'test_funct93f7c993ad3e89a2fb17716ef985a8a6.in.pl',
 'test_funct9af832e861123b84e12ff5dea30068af.in.pl',
 '.ipynb_checkpoints']

In [22]:
execute_function(test_funct, 1, 2)

3

In [23]:
os.listdir()

['store_function_v4.ipynb',
 'test_funct9af832e861123b84e12ff5dea30068af.out.pl',
 'store_function_v6.ipynb',
 'store_function_v2.ipynb',
 'store_function_v5.ipynb',
 'store_function_v1.ipynb',
 'store_function_v3.ipynb',
 'test_funct93f7c993ad3e89a2fb17716ef985a8a6.in.pl',
 'test_funct9af832e861123b84e12ff5dea30068af.in.pl',
 '.ipynb_checkpoints']

In [24]:
for f in os.listdir():
    if f.endswith(".pl"):
        os.remove(f)

In [25]:
os.listdir()

['store_function_v4.ipynb',
 'store_function_v6.ipynb',
 'store_function_v2.ipynb',
 'store_function_v5.ipynb',
 'store_function_v1.ipynb',
 'store_function_v3.ipynb',
 '.ipynb_checkpoints']